In [15]:
import pandas as pd 
import numpy as np 

In [16]:
#input data
#px: price; sz: size
data = pd.read_csv('../data/AAPL20250102_after.csv')
data.index = pd.to_datetime(data.index)
data = data.iloc[:2000]

In [ ]:
data.head(10)

In [18]:
#for loop to get the OFI data

# ask_sz = ['ask_sz_01', 'ask_sz_02', 'ask_sz_03', 'ask_sz_04', 'ask_sz_05']
# bid_sz = ['bid_sz_01', 'bid_sz_02', 'bid_sz_03', 'bid_sz_04', 'bid_sz_05']
# ask_px = ['ask_px_01', 'ask_px_02', 'ask_px_03', 'ask_px_04', 'ask_px_05']
# bid_px = ['bid_px_01', 'bid_px_02', 'bid_px_03', 'bid_px_04', 'bid_px_05']

# # Initialize the columns for bOF and aOF
# for i in range(1, 6):
#     data[f'bOF{i}'] = 0
#     data[f'aOF{i}'] = 0

# for i in range(1, 2000):
#     for j in range(5):
#         if data.iloc[i][bid_px[j]] > data.iloc[i-1][bid_px[j]]:
#             data.at[i, f'bOF{j + 1}'] = data.iloc[i][bid_sz[j]]
#         elif data.iloc[i][bid_px[j]] == data.iloc[i-1][bid_px[j]]:
#             data.at[i, f'bOF{j + 1}'] = data.iloc[i][bid_sz[j]] - data.iloc[i-1][bid_sz[j]]
#         else:
#             data.at[i, f'bOF{j + 1}'] = -data.iloc[i-1][bid_sz[j]]

#         if data.iloc[i][ask_px[j]] > data.iloc[i-1][ask_px[j]]:
#             data.at[i, f'aOF{j + 1}'] = -data.iloc[i-1][ask_sz[j]]
#         elif data.iloc[i][ask_px[j]] == data.iloc[i-1][ask_px[j]]:
#             data.at[i, f'aOF{j + 1}'] = data.iloc[i][ask_sz[j]] - data.iloc[i-1][ask_sz[j]]
#         else:
#             data.at[i, f'aOF{j + 1}'] = data.iloc[i][ask_sz[j]]



In [19]:

ask_sz = ['ask_sz_01', 'ask_sz_02', 'ask_sz_03', 'ask_sz_04', 'ask_sz_05']
bid_sz = ['bid_sz_01', 'bid_sz_02', 'bid_sz_03', 'bid_sz_04', 'bid_sz_05']
ask_px = ['ask_px_01', 'ask_px_02', 'ask_px_03', 'ask_px_04', 'ask_px_05']
bid_px = ['bid_px_01', 'bid_px_02', 'bid_px_03', 'bid_px_04', 'bid_px_05']

# 创建用于存储 bOF 和 aOF 的列
for i in range(1, 6):
    data[f'bOF{i}'] = 0
    data[f'aOF{i}'] = 0

# 计算 bid 和 ask 的价差矩阵
bid_px_diff = data[bid_px].diff().values  # bid_price 差值
ask_px_diff = data[ask_px].diff().values  # ask_price 差值

bid_sz_matrix = data[bid_sz].values
ask_sz_matrix = data[ask_sz].values
bid_sz_matrix_prev = np.roll(bid_sz_matrix, 1, axis=0)  # 滑动窗口的上一时刻值
ask_sz_matrix_prev = np.roll(ask_sz_matrix, 1, axis=0)

# 初始化 bOF 和 aOF 的矩阵
bOF_matrix = np.zeros_like(bid_px_diff)
aOF_matrix = np.zeros_like(ask_px_diff)

# 计算 bOF（买单订单流不平衡）
bOF_matrix[bid_px_diff > 0] = bid_sz_matrix[bid_px_diff > 0]  # bid价格上升
bOF_matrix[bid_px_diff == 0] = bid_sz_matrix[bid_px_diff == 0] - bid_sz_matrix_prev[bid_px_diff == 0]  # bid价格不变
bOF_matrix[bid_px_diff < 0] = -bid_sz_matrix_prev[bid_px_diff < 0]  # bid价格下降

# 计算 aOF（卖单订单流不平衡）
aOF_matrix[ask_px_diff > 0] = -ask_sz_matrix_prev[ask_px_diff > 0]  # ask价格上升
aOF_matrix[ask_px_diff == 0] = ask_sz_matrix[ask_px_diff == 0] - ask_sz_matrix_prev[ask_px_diff == 0]  # ask价格不变
aOF_matrix[ask_px_diff < 0] = ask_sz_matrix[ask_px_diff < 0]  # ask价格下降

# 将 bOF 和 aOF 的矩阵结果更新到 DataFrame
for j in range(5):
    data[f'bOF{j + 1}'] = bOF_matrix[:, j]
    data[f'aOF{j + 1}'] = aOF_matrix[:, j]


In [20]:
data['midprice'] = (data['ask_px_01'] + data['bid_px_01']) / 2

In [ ]:
data['last_size'] = data['size'].diff().values
data['last_size'][0] = data['size'][0]

In [ ]:
data[['size','last_size']].head(10) 

In [ ]:
freq = '1min'
data['event_count'] = 1
x = data.resample(freq
                    ).agg({
                            'OIBSH': 'sum', 'OIBNUM': 'sum', 'OIBDOL': 'sum',
                            'spread': 'mean',
                            'last': 'last', 'midprice': 'last',
                            'last_volume': 'sum',
                            'S_B_share': 'sum',

                            'aOF1': 'sum', 'aOF2': 'sum', 'aOF3': 'sum', 'aOF4': 'sum', 'aOF5': 'sum',
                            'bOF1': 'sum', 'bOF2': 'sum', 'bOF3': 'sum', 'bOF4': 'sum', 'bOF5': 'sum',


                            'a1_v': 'sum', 'a2_v': 'sum', 'a3_v': 'sum', 'a4_v': 'sum', 'a5_v': 'sum',
                            'b1_v': 'sum', 'b2_v': 'sum', 'b3_v': 'sum', 'b4_v': 'sum', 'b5_v': 'sum',

                            'event_count': 'sum'
                            }).rename(columns={'last_volume': 'volume'})
x['B-S/(BS)'] = x['OIBSH'] / x['S_B_share']
x = x.loc[~(x == 0).all(axis=1)]
x.dropna(inplace=True)

x['returns'] = np.log(x['midprice']) - np.log(x['midprice'].shift(1))
x['midprice_delta'] = x['midprice'] - x['midprice'].shift(1)


x['OFI1'] = x['bOF1'] - x['aOF1']
x['OFI2'] = x['bOF2'] - x['aOF2']
x['OFI3'] = x['bOF3'] - x['aOF3']
x['OFI4'] = x['bOF4'] - x['aOF4']
x['OFI5'] = x['bOF5'] - x['aOF5']

M = 5
Q1 = (x['a1_v'] + x['b1_v']) / 2 / x['event_count']
Q2 = (x['a2_v'] + x['b2_v']) / 2 / x['event_count']
Q3 = (x['a3_v'] + x['b3_v']) / 2 / x['event_count']
Q4 = (x['a4_v'] + x['b4_v']) / 2 / x['event_count']
Q5 = (x['a5_v'] + x['b5_v']) / 2 / x['event_count']

Qm = (Q1 + Q2 + Q3 + Q4 + Q5) / M
x['ofi1'] = x['OFI1'] / Qm
x['ofi2'] = x['OFI2'] / Qm
x['ofi3'] = x['OFI3'] / Qm
x['ofi4'] = x['OFI4'] / Qm
x['ofi5'] = x['OFI5'] / Qm

# x.dropna(inplace=True)
x = x[~x.isin([np.nan, np.inf, -np.inf]).any(axis=1)]
X = x[['ofi1', 'ofi2', 'ofi3', 'ofi4', 'ofi5']]
pca = PCA(n_components=1)
X_pca = pca.fit_transform(X)
ofiI = pd.Series(X_pca.flatten(), index=X.index)
standard_scaler = StandardScaler()
ofiI_standard = standard_scaler.fit_transform(np.array(ofiI).reshape(-1, 1))
x['ofiI'] = pd.Series(ofiI_standard.flatten(), index=X.index)

In [ ]:
data.head(10)

In [ ]:
def multi_imb_En_t(new_row: dict, last_row: dict, level: int = 5):
    """
    计算5个level档位的订单流不平衡
    """
    Wm = {'bOF1': 0, 'bOF2': 0, 'bOF3': 0, 'bOF4': 0, 'bOF5': 0}
    Vm = {'aOF1': 0, 'aOF2': 0, 'aOF3': 0, 'aOF4': 0, 'aOF5': 0}
    ask_v_key = ['a1_v', 'a2_v', 'a3_v', 'a4_v', 'a5_v']
    bid_v_key = ['b1_v', 'b2_v', 'b3_v', 'b4_v', 'b5_v']
    ask_p_key = ['a1', 'a2', 'a3', 'a4', 'a5']
    bid_p_key = ['b1', 'b2', 'b3', 'b4', 'b5']

    for i in range(0, level):
        if new_row[bid_p_key[i]] > last_row[bid_p_key[i]]:
            Wm[f'bOF{i + 1}'] = new_row[bid_v_key[i]]
        elif new_row[bid_p_key[i]] == last_row[bid_p_key[i]]:
            Wm[f'bOF{i + 1}'] = new_row[bid_v_key[i]] - last_row[bid_v_key[i]]
        else:
            Wm[f'bOF{i + 1}'] = -last_row[bid_v_key[i]]

        if new_row[ask_p_key[i]] > last_row[ask_p_key[i]]:
            Vm[f'aOF{i + 1}'] = -last_row[ask_v_key[i]]
        elif new_row[ask_p_key[i]] == last_row[ask_p_key[i]]:
            Vm[f'aOF{i + 1}'] = new_row[ask_v_key[i]] - last_row[ask_v_key[i]]
        else:
            Vm[f'aOF{i + 1}'] = new_row[ask_v_key[i]]

    # En = {f'En{k}': Wm[k] - Vm[k] for k in Wm}
    return Wm, Vm

def generate_data(data: pd.DataFrame):
    G_data = {}
    cal_footpoint_model = 'quota_method'
    last_row = []

    for index, row in tqdm(data.iterrows()):
        if not last_row:
            last_row = row

        G_data[index] = {}

        bOF, aOF = multi_imb_En_t(row, last_row)

        G_data[index].update(bOF)
        G_data[index].update(aOF)

        midpoint = (row['a1'] + row['b1']) / 2  # 计算中间价
        G_data[index].update({'midprice': midpoint})

        last_price = row['last']
        sub_data = {}
        if cal_footpoint_model == 'quota_method':
            if last_price > midpoint:
                pre_along = 1
            elif last_price < midpoint:
                pre_along = -1
            else:
                pre_along = pre_along

        sub_data['dirc'] = pre_along

        if cal_footpoint_model == 'quota_method':
            sub_data['OIBSH'] = pre_along * (max(0, row['volume'] - last_row['volume']))  # 股票时用的

        # sub_data['OIBNUM'] = pre_along * (row['num_trades'] - last_row['num_trades']) #　米筐的期货数据没有num_trades
        sub_data['OIBNUM'] = pre_along  # 没有num_trades时只能用每个tick当作一笔交易

        sub_data['spread'] = row['a1'] - row['b1']
        sub_data['S_B_share'] = max(0, row['volume'] - last_row['volume'])

        G_data[index].update(sub_data)
        G_data[index].update({'OIBDOL': pre_along * (max(0, row['total_turnover'] - last_row['total_turnover']))})
        G_data[index].update({'last_volume': max(0, row['volume'] - last_row['volume'])})  # 股票时用的

        last_row = row.to_dict()

    G_data = pd.DataFrame(G_data).T
    new_data = data.join(G_data)
    return new_data

def generate_target_freq_train_data(data: pd.DataFrame, freq: str):
    """
    :param freq 填入resample的频率  1min  15min  1D 等
    """
    freq = freq
    data['event_count'] = 1
    data['close'] = data['last']
    data['high'] = data['last']
    data['low'] = data['last']
    data['open'] = data['last']
    x = data.resample(freq
                      ).agg({'close': 'last', 'high': 'max', 'low': 'min', 'open': 'first',
                             'OIBSH': 'sum', 'OIBNUM': 'sum', 'OIBDOL': 'sum',
                             'spread': 'mean',
                             'last': 'last', 'midprice': 'last',
                             'last_volume': 'sum',
                             'S_B_share': 'sum',

                             'aOF1': 'sum', 'aOF2': 'sum', 'aOF3': 'sum', 'aOF4': 'sum', 'aOF5': 'sum',
                             'bOF1': 'sum', 'bOF2': 'sum', 'bOF3': 'sum', 'bOF4': 'sum', 'bOF5': 'sum',


                             'a1_v': 'sum', 'a2_v': 'sum', 'a3_v': 'sum', 'a4_v': 'sum', 'a5_v': 'sum',
                             'b1_v': 'sum', 'b2_v': 'sum', 'b3_v': 'sum', 'b4_v': 'sum', 'b5_v': 'sum',

                             'dirc': 'sum', 'event_count': 'sum'
                             }).rename(columns={'last_volume': 'volume'})
    x['B-S/(BS)'] = x['OIBSH'] / x['S_B_share']
    x = x.loc[~(x == 0).all(axis=1)]
    x.dropna(inplace=True)

    x['returns'] = np.log(x['midprice']) - np.log(x['midprice'].shift(1))
    x['midprice_delta'] = x['midprice'] - x['midprice'].shift(1)


    x['OFI1'] = x['bOF1'] - x['aOF1']
    x['OFI2'] = x['bOF2'] - x['aOF2']
    x['OFI3'] = x['bOF3'] - x['aOF3']
    x['OFI4'] = x['bOF4'] - x['aOF4']
    x['OFI5'] = x['bOF5'] - x['aOF5']

    M = 5
    Q1 = (x['a1_v'] + x['b1_v']) / 2 / x['event_count']
    Q2 = (x['a2_v'] + x['b2_v']) / 2 / x['event_count']
    Q3 = (x['a3_v'] + x['b3_v']) / 2 / x['event_count']
    Q4 = (x['a4_v'] + x['b4_v']) / 2 / x['event_count']
    Q5 = (x['a5_v'] + x['b5_v']) / 2 / x['event_count']

    Qm = (Q1 + Q2 + Q3 + Q4 + Q5) / M
    x['ofi1'] = x['OFI1'] / Qm
    x['ofi2'] = x['OFI2'] / Qm
    x['ofi3'] = x['OFI3'] / Qm
    x['ofi4'] = x['OFI4'] / Qm
    x['ofi5'] = x['OFI5'] / Qm

    # x.dropna(inplace=True)
    x = x[~x.isin([np.nan, np.inf, -np.inf]).any(axis=1)]
    X = x[['ofi1', 'ofi2', 'ofi3', 'ofi4', 'ofi5']]
    pca = PCA(n_components=1)
    X_pca = pca.fit_transform(X)
    ofiI = pd.Series(X_pca.flatten(), index=X.index)
    standard_scaler = StandardScaler()
    ofiI_standard = standard_scaler.fit_transform(np.array(ofiI).reshape(-1, 1))
    x['ofiI'] = pd.Series(ofiI_standard.flatten(), index=X.index)

    return x